<a href="https://colab.research.google.com/github/tienhuynh96/LLM_RLHF/blob/main/%5BDemo%5D_LLM_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Train LLM**

##**Instal library**

In [ ]:
!pip install -q transformers==4.38.2 tokenizers==0.15.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.3 MB/s eta 0:00:00


In [ ]:
# accelerate is a library by Hugging Face that provides utilities to help train and use models more efficiently across various hardware setups.
# -U is upgreade to the latest version
!pip install -q accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 21.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


##**Dataset**

In [ ]:
from datasets import load_dataset

# Load oscar english dataset
oscar_en = load_dataset("nthngdy/oscar-small", language='en')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for nthngdy/oscar-small contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nthngdy/oscar-small
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
oscar_en

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 595810
    })
})

In [ ]:
oscar_en['train']['text'][0]

'Mtendere Village was inspired by the vision of Chief Napoleon Dzombe, which he shared with John Blanchard during his first visit to Malawi. Chief Napoleon conveyed the desperate need for a program to intervene and care for the orphans and vulnerable children (OVC) in Malawi, and John committed to help.'

In [ ]:
# Filter the dataset to only include the first 1000 examples in the 'train' split.
oscar_en['train'] = oscar_en['train'].filter(lambda x: x['id'] < 1000)


In [ ]:
oscar_en

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 1000
    })
})

In [ ]:
# Write data to a text file
with open("oscar.en.txt", "w") as f:
    for text in oscar_en['train']['text']:
      f.write(text)

##**Tokenizer**

In [ ]:
# ByteLevelBPETokenizer tách từ theo các cặp ký tự
from tokenizers import ByteLevelBPETokenizer

paths = ['./oscar.en.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52000,
                min_frequency=2,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
                                ])

In [ ]:
# Save tokeinizer
!mkdir oscar_en
tokenizer.save_model("oscar_en")

['oscar_en/vocab.json', 'oscar_en/merges.txt']

In [ ]:
# Load vocabulary

from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./oscar_en/vocab.json",
    "./oscar_en/merges.txt",
)

In [ ]:
# Define post_processor class
# When call encode, model will add these added tokens
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
text_c = "I go to school"
tokenizer.encode(text_c).tokens

['<s>', 'I', 'Ġgo', 'Ġto', 'Ġschool', '</s>']

##**LM Training from Scratch**

In [ ]:
from transformers import RobertaConfig

# Set config to model
config = RobertaConfig(
    vocab_size=52000,
    max_position_embeddings=512,
    num_attention_heads=6,
    num_hidden_layers=4, # also known as transformer layers or encoder layers
    type_vocab_size=1  # does not need to distinguish between two different types of tokens within a single input sequence, the type_vocab_size in RoBERTa is often set to 1.
)

In [ ]:
from transformers import RobertaTokenizerFast
from transformers import RobertaForMaskedLM

# Set tokenizer and model
tokenizer = RobertaTokenizerFast.from_pretrained("./oscar_en", max_len=512)
model = RobertaForMaskedLM(config=config)

In [ ]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-3): 4 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [ ]:
model.num_parameters()

69327136

In [ ]:
from transformers import LineByLineTextDataset

# dataset is useful for tasks like language modeling where the input data is plain text, and each line in the text file is treated as a separate training example
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./oscar.en.txt",
    block_size=128, # Sets the maximum length of each tokenized sequence
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
next(iter(dataset))

{'input_ids': tensor([    0,  7372, 10209,   390,  9022,   412,   267,  6650,   294,  8680,
         10619,   368,    94,   299,  1150,    16,   553,   364,  5134,   348,
           896,  5639,  1072,   406,   884,  1658,   291,  3164,    18,  8680,
         10619,   385,   321,  1974,   267,  8857,   715,   330,   262,  1118,
           291,   694,   518,    73,   289,  1305,   330,   267, 10174,   289,
         10431,  1231,   359,    51,    58,    39,    13,   292,  3164,    16,
           289,   896,  2856,   291,   987,    18,    41,   337,  4044,   292,
          2756,   279,   294,   896,  1340,  8468,    93,   378,    87,  3650,
            16,  6540,  5639,    16,   422,  7100,  1118,   320,   787,  8492,
           291,   267,  5639,  1064,    18,  8323,  5639,    16,   379,  4298,
          8323,   387,  1087,   320,   547,  1941,   316,  9911,   291,   408,
          8220,    16,   306,  2828,   325,   789,  1072,   267,  3696,   478,
          3145,    16,   289,  1087,  6

In [ ]:
# create a data collator for language modeling
# Imports the DataCollatorForLanguageModeling class helps in preparing batches of data for language modeling tasks
from transformers import DataCollatorForLanguageModeling

# Set data_collator is for masked language modeling (MLM), which is a common pre-training task for models like BERT.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, # Set tokenizer
    mlm=True,            # Set MLM (masked language modeling) is True
    mlm_probability=0.15  # Set the probability of masking each token in the input sequence: 15%
)

In [ ]:
# Train model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./oscar_en",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=64,
    save_strategy='epoch',
    save_total_limit=2,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
%%time
trainer.train()

Step,Training Loss


CPU times: user 8.75 s, sys: 17 s, total: 25.8 s
Wall time: 1min 29s


TrainOutput(global_step=10, training_loss=10.228948974609375, metrics={'train_runtime': 89.1832, 'train_samples_per_second': 0.112, 'train_steps_per_second': 0.112, 'total_flos': 222698127360.0, 'train_loss': 10.228948974609375, 'epoch': 10.0})

##**Mask Token Prediction**

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
fill_mask("I go <mask> school.")

[{'score': 0.0002995965478476137,
  'token': 16,
  'token_str': ',',
  'sequence': 'I go, school.'},
 {'score': 0.00019186182180419564,
  'token': 10174,
  'token_str': ' orphans',
  'sequence': 'I go orphans school.'},
 {'score': 0.0001880171475932002,
  'token': 267,
  'token_str': ' the',
  'sequence': 'I go the school.'},
 {'score': 0.00018503617320675403,
  'token': 316,
  'token_str': 'ly',
  'sequence': 'I goly school.'},
 {'score': 0.0001592750777490437,
  'token': 17624,
  'token_str': '',
  'sequence': 'I go school.'}]